**Importing Libraries and ALGO**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

**Importing the Dataset**

In [2]:
red = pd.read_csv('winequality-red.csv', sep=';')
white = pd.read_csv('winequality-white.csv', sep=';')
red['wine_type'] = 'red'
white['wine_type'] = 'white'
data = pd.concat([red, white], ignore_index=True)

**Encoding**

In [3]:
le = LabelEncoder()
data['wine_type'] = le.fit_transform(data['wine_type'])

def classify_quality(score):
    if score <= 5:
        return 0
    elif score == 6:
        return 1
    else:
        return 2


data['quality_label'] = data['quality'].apply(classify_quality)
data.drop('quality', axis=1, inplace=True)

**Feature Engineering**

In [4]:
data['alcohol_pH'] = data['alcohol'] * data['pH']
data['sulphates_density'] = data['sulphates'] * data['density']

**Features & Target

In [7]:
X = data.drop('quality_label', axis=1)
y = data['quality_label']

scaler = PowerTransformer()
X_scaled = scaler.fit_transform(X)

**Train & test Splitting of dataset**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

**StackingEnsemble**


In [9]:
estimators = [
    ('lr', LogisticRegression(max_iter=500)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('svc', SVC(probability=True)),
    ('dt', DecisionTreeClassifier(max_depth=5)),
    ('rf', RandomForestClassifier(n_estimators=300, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, random_state=42)),
    ('xgb', XGBClassifier(n_estimators=300, learning_rate=0.1, max_depth=5, subsample=0.8, random_state=42))
]

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=RandomForestClassifier(n_estimators=200, random_state=42),
    cv=5,
    n_jobs=-1
)

**Train,Predict&Report**

In [10]:
stack_model.fit(X_train, y_train)


y_pred = stack_model.predict(X_test)


print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.76      0.77       477
           1       0.69      0.72      0.70       567
           2       0.73      0.66      0.70       256

    accuracy                           0.73      1300
   macro avg       0.73      0.72      0.72      1300
weighted avg       0.73      0.73      0.73      1300

